In [ ]:
import torch
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import copy
import matplotlib.pyplot as plt
import os
import random

seed=42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Normalisation des images pour les modèles pré-entraînés PyTorch

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
image_directory = "/content/gdrive/MyDrive/GitHub/ProjetFromage/data_fromage"
dataset_full = datasets.ImageFolder(image_directory, data_transforms)

# split en train, val et test à partir de la liste complète
np.random.seed(42)
samples_train, samples_test = train_test_split(dataset_full.samples)
samples_train, samples_val = train_test_split(samples_train,test_size=0.2)

print("Nombre d'images de train : %i" % len(samples_train))
print("Nombre d'images de val : %i" % len(samples_val))
print("Nombre d'images de test : %i" % len(samples_test))

Nombre d'images de train : 2288
Nombre d'images de val : 572
Nombre d'images de test : 954


In [ ]:
# on définit les datasets et loaders pytorch à partir des listes d'images de train / val / test
dataset_train = datasets.ImageFolder(image_directory, data_transforms)
dataset_train.samples = samples_train
dataset_train.imgs = samples_train
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=1)

dataset_val = datasets.ImageFolder(image_directory, data_transforms)
dataset_val.samples = samples_val
dataset_val.imgs = samples_val

dataset_test = datasets.ImageFolder(image_directory, data_transforms)
dataset_test.samples = samples_test
dataset_test.imgs = samples_test

torch.manual_seed(42)

In [ ]:
# détermination du nombre de classes
# vérification que les labels sont bien dans [0, nb_classes]
labels=[x[1] for x in samples_train]
if np.min(labels) != 0:
    print("Error: labels should start at 0 (min is %i)" % np.min(labels))
    sys.exit(-1)
if np.max(labels) != (len(np.unique(labels))-1):
    print("Error: labels should go from 0 to Nclasses (max label = {}; Nclasse = {})".format(np.max(labels),len(np.unique(labels)))  )
    sys.exit(-1)
nb_classes = np.max(labels)+1
print("Apprentissage sur {} classes".format(nb_classes))

Apprentissage sur 43 classes


In [ ]:
# on définit le modèle CNN
import math

NUM_CONV_2=64 
NUM_FC=1024 
NUM_CLASSES=nb_classes

def get_output_conv_2(kernel_size):
  H=224
  H1=math.floor(1+(H-(kernel_size-1)-1))
  H2=math.floor(1+(H1-2)/2)
  H3=math.floor(1+(H2-(5-1)-1))
  H4=math.floor(1+(H3-2)/2)
  return H4

class CNNNet(nn.Module):
    def __init__(self, kernel_size, num_conv_1):
        super(CNNNet,self).__init__()
        self.kernel_size_1=kernel_size
        self.conv_1 = nn.Conv2d(3,num_conv_1,kernel_size,1)
        self.conv_2 = nn.Conv2d(num_conv_1,NUM_CONV_2,5,1) # kernel_size = 5
        output_conv_2=get_output_conv_2(kernel_size)
        self.fc_1 = nn.Linear(output_conv_2*output_conv_2*NUM_CONV_2, NUM_FC)
        self.fc_2 = nn.Linear(NUM_FC,NUM_CLASSES)
    def forward(self,x):
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x,2,2)
        output_conv_2=get_output_conv_2(self.kernel_size_1)
        x = x.view(-1,output_conv_2*output_conv_2*NUM_CONV_2)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x


In [ ]:
# on définit le device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [ ]:
# on définit une fonction d'évaluation
def evaluate(model, dataset):
    avg_loss = 0.
    avg_accuracy = 0
    loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False, num_workers=2)
    for data in loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        n_correct = torch.sum(preds == labels)
        
        avg_loss += loss.item()
        avg_accuracy += n_correct
        
    return avg_loss / len(dataset), float(avg_accuracy) / len(dataset)

# fonction d'entraînement du modèle
PRINT_LOSS = False

def train_model(model, loader_train, data_val, optimizer, criterion, n_epochs=10):
  
    since = time.time()
    best_acc=0
    best_model_wts = copy.deepcopy(model.state_dict())
    saved_acc=[]

    for epoch in range(n_epochs): 
        batch=0
        print("EPOCH % i" % epoch)
        for i, data in enumerate(loader_train):
            batch+=1
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device) 
            optimizer.zero_grad() 
            outputs = model(inputs) 
            
            loss = criterion(outputs, labels) 
            if PRINT_LOSS:
                model.train(False)
                loss_val, accuracy = evaluate(model, data_val)
                model.train(True)
                #print("{} loss train: {:1.4f}\t val {:1.4f}\tAcc (val): {:.1%}".format(i, loss.item(), loss_val, accuracy   ))
            
            loss.backward() 
            optimizer.step() 
            #print(f'Batch {batch} : done')

        model.train(False)
        loss_val, accuracy = evaluate(model, data_val)
        saved_acc.append(accuracy)
        #early stopping
        if accuracy>best_acc:
          best_acc=accuracy
          best_model_wts = copy.deepcopy(model.state_dict())
        model.train(True)
        #print("{} loss train: {:1.4f}\t val {:1.4f}\tAcc (val): {:.1%}".format(i, loss.item(), loss_val, accuracy   ))

    model.load_state_dict(best_model_wts)
    return best_acc


In [ ]:
#===== Apprentissage

global_best_acc=0
best_param={}
best_net=CNNNet(3,16)

for kernel_size in [4,5,6]:
  for num_conv_1 in [16,32,48]:
    torch.cuda.empty_cache()
    my_net = CNNNet(kernel_size, num_conv_1)

    my_net.to(device) 
    my_net.train(True) 

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(my_net.parameters(), lr=0.001, momentum=0.9)

    print(f'Apprentissage : First Kernel Size = {kernel_size} ; Num conv 1 = {num_conv_1}')
    my_net.train(True)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    best_acc=train_model(my_net, loader_train, dataset_val, optimizer, criterion, n_epochs=10)
    print(f'Best Acc Val = {best_acc}\n')
    if best_acc>global_best_acc:
      global_best_acc=best_acc
      best_param={"First kernel size":kernel_size,"Num Conv 1":num_conv_1}
      best_net=my_net

print(">>> Best param :")
for key, item in best_param.items():
  print(f'{key} = {item}')

# évaluation
best_net.train(False)
loss, accuracy = evaluate(best_net, dataset_test)
print("Accuracy (test): %.1f%%" % (100 * accuracy))

Apprentissage : First Kernel Size = 4 ; Num conv 1 = 16
EPOCH  0
EPOCH  1
EPOCH  2
EPOCH  3
EPOCH  4
EPOCH  5
EPOCH  6
EPOCH  7
EPOCH  8
EPOCH  9
Best Acc Val = 0.40559440559440557

Apprentissage : First Kernel Size = 4 ; Num conv 1 = 32
EPOCH  0
EPOCH  1
EPOCH  2
EPOCH  3
EPOCH  4
EPOCH  5
EPOCH  6
EPOCH  7
EPOCH  8
EPOCH  9
Best Acc Val = 0.4160839160839161

Apprentissage : First Kernel Size = 4 ; Num conv 1 = 48
EPOCH  0
EPOCH  1
EPOCH  2
EPOCH  3
EPOCH  4
EPOCH  5
EPOCH  6
EPOCH  7
EPOCH  8
EPOCH  9
Best Acc Val = 0.43356643356643354

Apprentissage : First Kernel Size = 5 ; Num conv 1 = 16
EPOCH  0
EPOCH  1
EPOCH  2
EPOCH  3
EPOCH  4
EPOCH  5
EPOCH  6
EPOCH  7
EPOCH  8
EPOCH  9
Best Acc Val = 0.4230769230769231

Apprentissage : First Kernel Size = 5 ; Num conv 1 = 32
EPOCH  0
EPOCH  1
EPOCH  2
EPOCH  3
EPOCH  4
EPOCH  5
EPOCH  6
EPOCH  7
EPOCH  8
EPOCH  9
Best Acc Val = 0.42482517482517484

Apprentissage : First Kernel Size = 5 ; Num conv 1 = 48
EPOCH  0
EPOCH  1
EPOCH  2
EPOCH  3
